In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd GroundingDINO/
!pip install -e .

/content/drive/MyDrive
fatal: destination path 'GroundingDINO' already exists and is not an empty directory.
/content/drive/MyDrive/GroundingDINO
Obtaining file:///content/drive/MyDrive/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 19.8 MB/s eta 0:00:00
  Running setup.py develop for groundingdino


In [3]:
!mkdir /content/drive/MyDrive/GroundingDINO/weights
%cd /content/drive/MyDrive/GroundingDINO/weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

mkdir: cannot create directory ‘/content/drive/MyDrive/GroundingDINO/weights’: File exists
/content/drive/MyDrive/GroundingDINO/weights


In [4]:
%cd /content/drive/MyDrive/GroundingDINO

/content/drive/MyDrive/GroundingDINO


In [20]:
import os
import typer
import cv2
import torch
import torchvision
from groundingdino.util.inference import load_model, load_image, predict, annotate
from tqdm import tqdm

text_prompt = 'warehouse pallets, pallet, wooden pallet, ground, floor'
box_threshold = 0.3
text_threshold = 0.35
export_annotated_images = True
config_path="/content/drive/MyDrive/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
weights_path="/content/drive/MyDrive/GroundingDINO/weights/groundingdino_swint_ogc.pth"
model = load_model(config_path, weights_path)

final text_encoder_type: bert-base-uncased


In [6]:
import os
import cv2
import torchvision.ops
from tqdm import tqdm
import torch

image_directory = '/content/drive/MyDrive/Pallets'
dataset = os.listdir(image_directory)

In [13]:
# Define a function to convert bounding box coordinates from PascalVOC to YOLO format
def convert_to_yolo_format(xmin, ymin, xmax, ymax, img_width, img_height):
    x_center = ((xmin + xmax) / 2) / img_width
    y_center = ((ymin + ymax) / 2) / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height
    return x_center, y_center, width, height

# Define a function to save YOLO annotations
def save_yolo_txt(image_filename, image_shape, boxes, labels, output_dir):
    img_width, img_height = image_shape[1], image_shape[0]
    yolo_annotation_path = os.path.join(output_dir, f"{os.path.splitext(image_filename)[0]}.txt")

    with open(yolo_annotation_path, "w") as file:
        for box, label in zip(boxes, labels):
            xmin, ymin, xmax, ymax = box
            x_center, y_center, width, height = convert_to_yolo_format(xmin, ymin, xmax, ymax, img_width, img_height)
            print(label)
            if label not in ['ground','floor']:
              label = '0'
              file.write(f"{label} {x_center} {y_center} {width} {height}\n")
            else  :
              label = '1'
              file.write(f"{label} {x_center} {y_center} {width} {height}\n")

In [21]:
# Initialize a directory for saving YOLO annotations
yolo_annotation_dir = os.path.join("/content/drive/MyDrive/Pallet_Annotation", "yolo_annotations")
annotated_dir = os.path.join('/content/drive/MyDrive', "annotated_images_det")
os.makedirs(yolo_annotation_dir, exist_ok=True)
model = model.to("cuda")
# Iterate through your dataset
for image_filename in tqdm(dataset):
    image_path = os.path.join(image_directory, image_filename)
    image_source, image = load_image(image_path)
    image = image.to(torch.device("cuda")).float()

    # Obtain model predictions
    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption=text_prompt,
        box_threshold=box_threshold,
        text_threshold=text_threshold,
        device="cuda"
    )

    # Get image dimensions and convert boxes
    c, h, w = image.shape
    boxes_rs = boxes * torch.Tensor([w, h, w, h])
    rel_box = torchvision.ops.box_convert(boxes=boxes_rs, in_fmt="cxcywh", out_fmt="xyxy").numpy()

    # Save the YOLO annotations for this image
    save_yolo_txt(image_filename=image_filename, image_shape=[h, w, c], boxes=rel_box, labels=phrases, output_dir=yolo_annotation_dir)

    # Save the annotated image with bounding boxes
    output_file = os.path.join(annotated_dir, f"{os.path.splitext(image_filename)[0]}_annotated.jpg")
    annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
    cv2.imwrite(output_file, annotated_frame)

print(f"YOLO annotations saved to {yolo_annotation_dir}")

  0%|          | 2/519 [00:02<09:57,  1.16s/it]

floor
warehouse pallets
ground

floor

ground
pallet


  1%|          | 4/519 [00:03<04:10,  2.06it/s]

warehouse pallets
wooden pallet
pallet
floor
pallets
pallet








  1%|          | 6/519 [00:03<02:30,  3.40it/s]

...



pallet
wooden pallet

floor
floor







100%|█████████▉| 517/519 [01:23<00:00,  6.48it/s]

floor
warehouse pallets
pallets


##let
ground


floor
warehouse pallets
pallets
ground



100%|██████████| 519/519 [01:23<00:00,  6.22it/s]

floor





ground

floor

warehouse pallets


YOLO annotations saved to /content/drive/MyDrive/Pallet_Annotation/yolo_annotations


In [ ]:
!git clone https://github.com/facebookresearch/segment-anything.git
%cd segment-anything


fatal: destination path 'segment-anything' already exists and is not an empty directory.
/content/segment-anything


In [ ]:
!pip install -r requirements.txt
!pip install .

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Processing /content/segment-anything
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=b10f8d0d0828998b6975d99ff15581f2dfd27af579e0be0ae58c9ca73f29350a
  Stored in directory: /root/.cache/pip/wheels/b4/ba/0a/26204d95149815c291179f15d5f40f59a4f6ffa79e5bd1ccc2
Successfully built segment_anything


In [ ]:
import os
import cv2
import numpy as np
import torch
from segment_anything import SamPredictor, sam_model_registry
from torchvision.ops import box_convert
from PIL import Image

# Initialize SAM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam_model_type = "vit_l"  # Choose the model type (e.g., vit_b, vit_l, vit_h)
sam_checkpoint = "/content/drive/MyDrive/sam_vit_l_0b3195.pth"  # Path to SAM model weights
sam = sam_model_registry[sam_model_type](checkpoint=sam_checkpoint).to(device)
predictor = SamPredictor(sam)

# Paths to images and YOLO annotations
image_dir = "/content/drive/MyDrive/Pallets"  # Directory with input images
annotation_dir = "/content/drive/MyDrive/Pallet_Annotation/yolo_annotations_enhanced"  # Directory with YOLO annotations
output_dir = "/content/drive/MyDrive/seg_out_enhanced"  # Output directory for segmentation maps
os.makedirs(output_dir, exist_ok=True)

# Function to load YOLO annotations
def load_yolo_boxes(annotation_file, img_width, img_height):
    boxes = []
    with open(annotation_file, "r") as f:
        for line in f:
            try :
              parts = line.strip().split()

            # Skip the first part (label text) if it's non-numeric, and parse the coordinates
              if not parts[0].isdigit():
                  parts = parts[1:]  # Skip the label if it is text like 'pallet'

              x_center, y_center, width, height = map(float,parts)
              # Convert YOLO format (x_center, y_center, width, height) to bounding box (xmin, ymin, xmax, ymax)
              x_min = max(0,int((x_center - width / 2) * img_width))
              y_min = max(0,int((y_center - height / 2) * img_height))
              x_max = int((x_center + width / 2) * img_width)
              y_max = int((y_center + height / 2) * img_height)
              if x_max > x_min and y_max > y_min:
                boxes.append([x_min, y_min, x_max, y_max])
              else :
                print("Empty boxes : ", annotation_file)
            except ValueError:
                print("Value error: ",annotation_file)
                continue
    return boxes

count = 0
# Process each image and its corresponding annotation
for image_filename in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_filename)
    annotation_path = os.path.join(annotation_dir, os.path.splitext(image_filename)[0] + ".txt")

    # Load image
    image = np.array(Image.open(image_path).convert("RGB"))
    img_height, img_width = image.shape[:2]

    # Load YOLO bounding boxes
    if not os.path.exists(annotation_path):
        print("Annotation not found : ", annotation_path)
        continue
    boxes = load_yolo_boxes(annotation_path, img_width, img_height)

    # Prepare image for SAM
    predictor.set_image(image)
    boxes = np.array(boxes)
    combined_mask = np.zeros((img_height, img_width), dtype=np.uint8)

    # Generate segmentation masks for each bounding box
    for box in boxes:
        # Use SAM to predict the mask within each bounding box
        mask, _, _ = predictor.predict(
            box=box,
            multimask_output=False
        )

        mask = mask[0].astype(np.uint8) * 255  # Convert boolean mask to binary (0 or 255)

        # Resize the mask to fit the bounding box location within the combined mask
        x_min, y_min, x_max, y_max = box
        resized_mask = cv2.resize(mask, (x_max - x_min, y_max - y_min))

        # Place the resized mask into the correct location in the combined mask
        combined_mask[y_min:y_max, x_min:x_max] = np.maximum(combined_mask[y_min:y_max, x_min:x_max], resized_mask)

        # Save the segmentation mask
    output_filename = os.path.join(output_dir, f"{os.path.splitext(image_filename)[0]}.png")
    count+=1
    cv2.imwrite(output_filename, combined_mask)

print(f"Segmentation maps saved in {output_dir}")
print(count)

/content/segment-anything/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Segmentation maps saved in /content/drive/MyDrive/seg_out_enhanced
519


In [ ]:
count, count1 = 0,0
for image_filename in os.listdir('/content/drive/MyDrive/seg_out_enhanced'):
  count+=1

for image_filename in os.listdir('/content/drive/MyDrive/Pallets'):
  count1+=1

print("Input images : ",count1)
print("Segmented images : ", count)


Input images :  519
Segmented images :  519


In [ ]:
import os

# Define the directory containing the .txt files
directory = '/content/drive/MyDrive/Pallet_Annotation/yolo_annotations_enhanced'

# Loop through each file in the directory
for filename in os.listdir(directory):
    # Check if the file is a .txt file
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)

        # Open and read the file
        with open(file_path, 'r') as file:
            content = file.read()

        # Replace 'pallet' with '1'
        new_content = content.replace('pallet', '0')

        # Write the modified content back to the file
        with open(file_path, 'w') as file:
            file.write(new_content)



print("Replacement completed.")


Replacement completed.
